### Chargement des bibliothèque et des dataset

In [124]:

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import datetime



import pytz
# Utilisation du module datetime
now =datetime.datetime.now()

### Data set des trades

In [125]:
df=pd.read_csv(r"C:\Users\podoG\Downloads\trades_history_btc.csv")

df = df.dropna(subset=['datetime'])
df

,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime
0,1675164424534,2599691362,18016256435,22830.92,0.00066,0.000000e+00,BNB,True,False,2023-01-31 11:27:04
1,1675164424534,2599691363,18016256435,22831.05,0.00064,0.000000e+00,BNB,True,False,2023-01-31 11:27:04
2,1675164424534,2599691364,18016256435,22831.08,0.00100,0.000000e+00,BNB,True,False,2023-01-31 11:27:04
3,1675164424534,2599691365,18016256435,22831.28,0.03032,0.000000e+00,BNB,True,False,2023-01-31 11:27:04
4,1675205405138,2603428090,18037171280,23147.74,0.00064,0.000000e+00,BNB,True,False,2023-01-31 22:50:05
...,...,...,...,...,...,...,...,...,...,...
69,1681639267480,3082923341,20813706743,30275.40,0.00061,6.100000e-07,BTC,True,False,2023-04-16 10:01:07
70,1681700412716,3083612606,20819248621,30014.64,0.03091,9.277525e-01,USDT,False,False,2023-04-17 03:00:12
71,1681834866318,3085329058,20833800173,30245.99,0.03004,3.004000e-05,BTC,True,False,2023-04-18 16:21:06
72,1681834868345,3085329061,20833800272,30245.99,0.00060,6.000000e-07,BTC,True,False,2023-04-18 16:21:08


In [126]:

df.isna().sum()

timestamp           0
id                  0
order_id            0
price               0
qty                 0
commission          0
commission_asset    0
is_buyer            0
is_maker            0
datetime            0
dtype: int64

### Identification de l'id comme point de depart temporel pour analyse du bot

In [127]:


df['direction'] = np.where(df['is_buyer'], -1, 1)
df = df.loc[df['id'] >= 3055669527]
df

,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime,direction
12,1679671967140,3055669527,20582103493,27910.05,0.03205,8.945171e-01,USDT,False,True,2023-03-24 15:32:47,1
13,1679705060369,3056431195,20588276078,27567.41,0.03242,3.242000e-05,BTC,True,False,2023-03-25 00:44:20,-1
14,1679936404645,3059762994,20618072666,27135.79,0.00038,1.031160e-02,USDT,False,False,2023-03-27 17:00:04,1
15,1679936404645,3059762995,20618072666,27135.79,0.00643,1.744831e-01,USDT,False,False,2023-03-27 17:00:04,1
16,1679936404645,3059762996,20618072666,27135.79,0.00642,1.742118e-01,USDT,False,False,2023-03-27 17:00:04,1
...,...,...,...,...,...,...,...,...,...,...,...
69,1681639267480,3082923341,20813706743,30275.40,0.00061,6.100000e-07,BTC,True,False,2023-04-16 10:01:07,-1
70,1681700412716,3083612606,20819248621,30014.64,0.03091,9.277525e-01,USDT,False,False,2023-04-17 03:00:12,1
71,1681834866318,3085329058,20833800173,30245.99,0.03004,3.004000e-05,BTC,True,False,2023-04-18 16:21:06,-1
72,1681834868345,3085329061,20833800272,30245.99,0.00060,6.000000e-07,BTC,True,False,2023-04-18 16:21:08,-1


In [128]:
# Créer une colonne Wallet à partir de qty et price
df['Wallet'] = df['qty'] * df['price']


C:\Users\podoG\AppData\Local\Temp\ipykernel_21848\2175154003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Wallet'] = df['qty'] * df['price']



### création d'un index de trade

In [129]:
# Initialisation pour la création d'un index de trade
current_trade_id = 0
current_trade_is_buyer = None

# Parcours du dataset
for index, row in df.iterrows():
    if row['is_buyer'] and current_trade_is_buyer != True:
        # Nouveau trade, on met à jour l'ID et le sens de la transaction
        current_trade_id += 1
        current_trade_is_buyer = True
    elif not row['is_buyer'] and current_trade_is_buyer != False:
        # Fin du trade, on met à jour le sens de la transaction
        current_trade_is_buyer = False
    # Ajout de l'ID unique pour le trade en cours
    df.loc[index, 'trade_id'] = current_trade_id

C:\Users\podoG\AppData\Local\Temp\ipykernel_21848\1100707956.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'trade_id'] = current_trade_id


In [130]:
# suppression d'un trade pour avoir le bon point de départ
df.drop(12, inplace=True)
df

C:\Users\podoG\AppData\Local\Temp\ipykernel_21848\2375241884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(12, inplace=True)


,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime,direction,Wallet,trade_id
13,1679705060369,3056431195,20588276078,27567.41,0.03242,3.242000e-05,BTC,True,False,2023-03-25 00:44:20,-1,893.735432,1.0
14,1679936404645,3059762994,20618072666,27135.79,0.00038,1.031160e-02,USDT,False,False,2023-03-27 17:00:04,1,10.311600,1.0
15,1679936404645,3059762995,20618072666,27135.79,0.00643,1.744831e-01,USDT,False,False,2023-03-27 17:00:04,1,174.483130,1.0
16,1679936404645,3059762996,20618072666,27135.79,0.00642,1.742118e-01,USDT,False,False,2023-03-27 17:00:04,1,174.211772,1.0
17,1679936404645,3059762997,20618072666,27135.78,0.00067,1.818097e-02,USDT,False,False,2023-03-27 17:00:04,1,18.180973,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1681639267480,3082923341,20813706743,30275.40,0.00061,6.100000e-07,BTC,True,False,2023-04-16 10:01:07,-1,18.467994,14.0
70,1681700412716,3083612606,20819248621,30014.64,0.03091,9.277525e-01,USDT,False,False,2023-04-17 03:00:12,1,927.752522,14.0
71,1681834866318,3085329058,20833800173,30245.99,0.03004,3.004000e-05,BTC,True,False,2023-04-18 16:21:06,-1,908.589540,15.0
72,1681834868345,3085329061,20833800272,30245.99,0.00060,6.000000e-07,BTC,True,False,2023-04-18 16:21:08,-1,18.147594,15.0


In [131]:
# Création d'un dictionnaire pour stocker les timestamps start et stop de chaque trade
trades_dict = {}

# Parcours de chaque trade
for trade_id, group in df.groupby('trade_id'):
    # Recherche de la première ligne du trade (direction=-1)
    entry_index = group[group['direction'] == -1].index[0]
    # Recherche de la dernière ligne du trade (direction=1)
    exit_index = group[group['direction'] == 1].index[-1]
    # Stockage des timestamps start et stop dans le dictionnaire
    trades_dict[trade_id] = {'timestamp_start': group.loc[entry_index]['timestamp'], 'timestamp_stop': group.loc[exit_index]['timestamp']}

# Création du DataFrame trades_df à partir du dictionnaire
trades_df = pd.DataFrame.from_dict(trades_dict, orient='index').reset_index().rename(columns={'index': 'trade_id'})


In [132]:
trades_df

,trade_id,timestamp_start,timestamp_stop
0,1.0,1679705060369,1679936404645
1,2.0,1679990404901,1680123604404
2,3.0,1680248763230,1680248764795
3,4.0,1680285605376,1680354006451
4,5.0,1680418806067,1680458406234
5,6.0,1680526805716,1680573606150
6,7.0,1680616805575,1680724805870
7,8.0,1680807605810,1680872405953
8,9.0,1680912005663,1680994805795
9,10.0,1681084805640,1681142406399


In [133]:
# ON FAIT UN DF DE TOKENQTY QUI SERA UTILISER APRES
df_qty = df[df['direction'] == -1].groupby('trade_id').agg({'qty': 'sum'}).reset_index()
df_qty = df_qty.rename(columns={'qty': 'token_qty'})
df_qty


# ON FAIT UN DF DE usdqtY QUI SERA UTILISER APRES
df_qty_usdt = df[df['direction'] == 1].groupby('trade_id').agg({'Wallet': 'sum'}).reset_index()
df_qty_usdt = df_qty_usdt.rename(columns={'Wallet': 'usdt_qty'})


# on cree un df_init pour chaque trade id , l'idée et de stocker le volume usdt au départ du trade de chaque id
df_init= df[df['direction'] == -1].groupby('trade_id').agg({'Wallet': 'sum'}).reset_index()

df_qty


,trade_id,token_qty
0,1.0,0.03242
1,2.0,0.03254
2,3.0,0.03343
3,4.0,0.03248
4,5.0,0.03236
5,6.0,0.03234
6,7.0,0.03190
7,8.0,0.03216
8,9.0,0.03208
9,10.0,0.03157


In [134]:
df_init # on réutilisera plus tard pour avoir un suivie du drawdown 

,trade_id,Wallet
0,1.0,893.735432
1,2.0,877.376084
2,3.0,922.548989
3,4.0,920.304645
4,5.0,921.029910
5,6.0,910.666151
6,7.0,897.772308
7,8.0,898.993397
8,9.0,895.235040
9,10.0,894.220023


In [135]:
df_qty_usdt

,trade_id,usdt_qty
0,1.0,878.631065
1,2.0,922.912736
2,3.0,921.639266
3,4.0,922.038227
4,5.0,911.524629
5,6.0,898.740130
6,7.0,899.917652
7,8.0,896.189881
8,9.0,895.126151
9,10.0,896.978045


In [136]:
df_info = pd.merge(trades_df, df_qty, on='trade_id', how='left')
df_info=pd.merge(df_info,df_qty_usdt,on='trade_id',how='left')
df_info=pd.merge(df_info,df_init,on='trade_id',how='left')
df_info

,trade_id,timestamp_start,timestamp_stop,token_qty,usdt_qty,Wallet
0,1.0,1679705060369,1679936404645,0.03242,878.631065,893.735432
1,2.0,1679990404901,1680123604404,0.03254,922.912736,877.376084
2,3.0,1680248763230,1680248764795,0.03343,921.639266,922.548989
3,4.0,1680285605376,1680354006451,0.03248,922.038227,920.304645
4,5.0,1680418806067,1680458406234,0.03236,911.524629,921.029910
5,6.0,1680526805716,1680573606150,0.03234,898.740130,910.666151
6,7.0,1680616805575,1680724805870,0.03190,899.917652,897.772308
7,8.0,1680807605810,1680872405953,0.03216,896.189881,898.993397
8,9.0,1680912005663,1680994805795,0.03208,895.126151,895.235040
9,10.0,1681084805640,1681142406399,0.03157,896.978045,894.220023


In [137]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 13 to 73
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   timestamp         61 non-null     int64  
 1   id                61 non-null     int64  
 2   order_id          61 non-null     int64  
 3   price             61 non-null     float64
 4   qty               61 non-null     float64
 5   commission        61 non-null     float64
 6   commission_asset  61 non-null     object 
 7   is_buyer          61 non-null     bool   
 8   is_maker          61 non-null     bool   
 9   datetime          61 non-null     object 
 10  direction         61 non-null     int32  
 11  Wallet            61 non-null     float64
 12  trade_id          61 non-null     float64
dtypes: bool(2), float64(5), int32(1), int64(3), object(2)
memory usage: 5.6+ KB


In [138]:
df['trade_number'] = df.groupby('trade_id').cumcount() + 1
df['temp_profit'] = df['Wallet'] * df['direction']

df

C:\Users\podoG\AppData\Local\Temp\ipykernel_21848\2115697901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trade_number'] = df.groupby('trade_id').cumcount() + 1
C:\Users\podoG\AppData\Local\Temp\ipykernel_21848\2115697901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_profit'] = df['Wallet'] * df['direction']


,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime,direction,Wallet,trade_id,trade_number,temp_profit
13,1679705060369,3056431195,20588276078,27567.41,0.03242,3.242000e-05,BTC,True,False,2023-03-25 00:44:20,-1,893.735432,1.0,1,-893.735432
14,1679936404645,3059762994,20618072666,27135.79,0.00038,1.031160e-02,USDT,False,False,2023-03-27 17:00:04,1,10.311600,1.0,2,10.311600
15,1679936404645,3059762995,20618072666,27135.79,0.00643,1.744831e-01,USDT,False,False,2023-03-27 17:00:04,1,174.483130,1.0,3,174.483130
16,1679936404645,3059762996,20618072666,27135.79,0.00642,1.742118e-01,USDT,False,False,2023-03-27 17:00:04,1,174.211772,1.0,4,174.211772
17,1679936404645,3059762997,20618072666,27135.78,0.00067,1.818097e-02,USDT,False,False,2023-03-27 17:00:04,1,18.180973,1.0,5,18.180973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1681639267480,3082923341,20813706743,30275.40,0.00061,6.100000e-07,BTC,True,False,2023-04-16 10:01:07,-1,18.467994,14.0,2,-18.467994
70,1681700412716,3083612606,20819248621,30014.64,0.03091,9.277525e-01,USDT,False,False,2023-04-17 03:00:12,1,927.752522,14.0,3,927.752522
71,1681834866318,3085329058,20833800173,30245.99,0.03004,3.004000e-05,BTC,True,False,2023-04-18 16:21:06,-1,908.589540,15.0,1,-908.589540
72,1681834868345,3085329061,20833800272,30245.99,0.00060,6.000000e-07,BTC,True,False,2023-04-18 16:21:08,-1,18.147594,15.0,2,-18.147594


In [139]:
df.groupby("trade_id")["temp_profit"].sum()



trade_id
1.0    -15.104367
2.0     45.536652
3.0     -0.909723
4.0      1.733582
5.0     -9.505281
6.0    -11.926022
7.0      2.145344
8.0     -2.803517
9.0     -0.108889
10.0     2.758022
11.0    31.914508
12.0    12.605594
13.0    -1.189219
14.0    -9.081181
15.0    -0.225389
Name: temp_profit, dtype: float64

In [140]:
profits = df.groupby("trade_id")["temp_profit"].sum().reset_index(name="profits")
df = df.merge(profits, on="trade_id")
df

,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime,direction,Wallet,trade_id,trade_number,temp_profit,profits
0,1679705060369,3056431195,20588276078,27567.41,0.03242,3.242000e-05,BTC,True,False,2023-03-25 00:44:20,-1,893.735432,1.0,1,-893.735432,-15.104367
1,1679936404645,3059762994,20618072666,27135.79,0.00038,1.031160e-02,USDT,False,False,2023-03-27 17:00:04,1,10.311600,1.0,2,10.311600,-15.104367
2,1679936404645,3059762995,20618072666,27135.79,0.00643,1.744831e-01,USDT,False,False,2023-03-27 17:00:04,1,174.483130,1.0,3,174.483130,-15.104367
3,1679936404645,3059762996,20618072666,27135.79,0.00642,1.742118e-01,USDT,False,False,2023-03-27 17:00:04,1,174.211772,1.0,4,174.211772,-15.104367
4,1679936404645,3059762997,20618072666,27135.78,0.00067,1.818097e-02,USDT,False,False,2023-03-27 17:00:04,1,18.180973,1.0,5,18.180973,-15.104367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1681639267480,3082923341,20813706743,30275.40,0.00061,6.100000e-07,BTC,True,False,2023-04-16 10:01:07,-1,18.467994,14.0,2,-18.467994,-9.081181
57,1681700412716,3083612606,20819248621,30014.64,0.03091,9.277525e-01,USDT,False,False,2023-04-17 03:00:12,1,927.752522,14.0,3,927.752522,-9.081181
58,1681834866318,3085329058,20833800173,30245.99,0.03004,3.004000e-05,BTC,True,False,2023-04-18 16:21:06,-1,908.589540,15.0,1,-908.589540,-0.225389
59,1681834868345,3085329061,20833800272,30245.99,0.00060,6.000000e-07,BTC,True,False,2023-04-18 16:21:08,-1,18.147594,15.0,2,-18.147594,-0.225389


In [141]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 0 to 60
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   timestamp         61 non-null     int64  
 1   id                61 non-null     int64  
 2   order_id          61 non-null     int64  
 3   price             61 non-null     float64
 4   qty               61 non-null     float64
 5   commission        61 non-null     float64
 6   commission_asset  61 non-null     object 
 7   is_buyer          61 non-null     bool   
 8   is_maker          61 non-null     bool   
 9   datetime          61 non-null     object 
 10  direction         61 non-null     int32  
 11  Wallet            61 non-null     float64
 12  trade_id          61 non-null     float64
 13  trade_number      61 non-null     int64  
 14  temp_profit       61 non-null     float64
 15  profits           61 non-null     float64
dtypes: bool(2), float64(7), int32(1), int64(4), ob

In [142]:
df["profits"] = df["profits"].drop_duplicates()
df=df.fillna(0)


In [143]:
# Calcul de la somme des temp_profit avec direction = -1 pour chaque trade_id
temp_profit_sum = df.loc[df['direction'] == -1].groupby('trade_id')['temp_profit'].sum()

# Inverser le signe de la somme des temp_profit pour que le signe corresponde à celui des profits
temp_profit_sum = -1 * temp_profit_sum

# Calcul de la colonne profits_ratio pour chaque trade_id
df['profits_ratio'] = df['profits'] / df['trade_id'].map(temp_profit_sum) * 100
df['temp_profit_sum']=df['trade_id'].map(temp_profit_sum)
# Affichage du DataFrame avec la colonne profits_ratio
df.head()

,timestamp,id,order_id,price,qty,commission,commission_asset,is_buyer,is_maker,datetime,direction,Wallet,trade_id,trade_number,temp_profit,profits,profits_ratio,temp_profit_sum
0,1679705060369,3056431195,20588276078,27567.41,0.03242,0.000032,BTC,True,False,2023-03-25 00:44:20,-1,893.735432,1.0,1,-893.735432,-15.104367,-1.690027,893.735432
1,1679936404645,3059762994,20618072666,27135.79,0.00038,0.010312,USDT,False,False,2023-03-27 17:00:04,1,10.311600,1.0,2,10.311600,0.000000,0.000000,893.735432
2,1679936404645,3059762995,20618072666,27135.79,0.00643,0.174483,USDT,False,False,2023-03-27 17:00:04,1,174.483130,1.0,3,174.483130,0.000000,0.000000,893.735432
3,1679936404645,3059762996,20618072666,27135.79,0.00642,0.174212,USDT,False,False,2023-03-27 17:00:04,1,174.211772,1.0,4,174.211772,0.000000,0.000000,893.735432
4,1679936404645,3059762997,20618072666,27135.78,0.00067,0.018181,USDT,False,False,2023-03-27 17:00:04,1,18.180973,1.0,5,18.180973,0.000000,0.000000,893.735432


### Chargement du deuxieme dataset ( historique de cour)

In [149]:
# pour calculer le drawdown il nous faut l'historique du token sur la période équivalente
# -- Import --
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# -- Define Binance Client --
client = Client()

# -- You can change the crypto pair ,the start date and the time interval below --
pairName = "BTCUSDT"
startDate = " 24 mars 2023"
timeInterval = Client.KLINE_INTERVAL_1HOUR



# -- Load all price data from binance API --
klinesT = client.get_historical_klines(pairName, timeInterval, startDate)





# -- Define your dataset --
df_h = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df_h['close'] = pd.to_numeric(df_h['close'])
df_h['high'] = pd.to_numeric(df_h['high'])
df_h['low'] = pd.to_numeric(df_h['low'])
df_h['open'] = pd.to_numeric(df_h['open'])

# -- Set the date to index --

df_h['datetime']= pd.to_datetime(df_h.timestamp, unit='ms')

print("Data loaded 100%")

# -- Uncomment the line below if you want to check your price dataset --
df_h

Data loaded 100%


,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore,datetime
0,1679616000000,28295.42,28374.30,28182.02,28195.30,2887.21044000,1679619599999,81696702.94330860,55358,1361.90716000,38542337.80235720,0,2023-03-24 00:00:00
1,1679619600000,28195.29,28318.80,28137.93,28253.00,2294.16851000,1679623199999,64796470.41189460,43247,1199.22863000,33870352.85455300,0,2023-03-24 01:00:00
2,1679623200000,28253.00,28309.99,28134.86,28186.28,2066.26067000,1679626799999,58366428.14922550,37281,889.81155000,25135233.48266430,0,2023-03-24 02:00:00
3,1679626800000,28186.28,28237.10,28158.40,28219.98,1572.69903000,1679630399999,44359314.47674330,30487,800.65146000,22581411.60296640,0,2023-03-24 03:00:00
4,1679630400000,28219.98,28254.48,28102.15,28130.41,2009.41868000,1679633999999,56646713.28924120,36868,943.61911000,26602825.20375980,0,2023-03-24 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,1682064000000,28155.02,28237.30,28015.71,28071.90,2847.93515000,1682067599999,80064735.18189570,57792,1389.19303000,39057693.08022460,0,2023-04-21 08:00:00
680,1682067600000,28071.89,28112.50,27967.09,28060.85,1945.42052000,1682071199999,54560381.19529480,37520,851.81226000,23889305.67887860,0,2023-04-21 09:00:00
681,1682071200000,28060.85,28152.62,27936.80,28001.96,2090.75767000,1682074799999,58652886.13923020,42506,997.57601000,27987915.97858730,0,2023-04-21 10:00:00
682,1682074800000,28001.96,28093.55,27919.69,28066.21,2602.24007000,1682078399999,72880442.44866950,45978,1336.73757000,37437351.66433840,0,2023-04-21 11:00:00


### Fusion des dataset

In [150]:


# Charger les deux dataframes
df1 = df
df2 = df_h

# Concaténer les deux dataframes et trier par le timestamp
df = pd.concat([df1, df2], axis=0, sort=True).reset_index(drop=True)

# Afficher les 25 premières lignes triées par timestamp
df_sorted = df.sort_values(by='timestamp')
df_sorted.head()


,Wallet,close,close_time,commission,commission_asset,datetime,direction,high,id,ignore,...,quote_av,tb_base_av,tb_quote_av,temp_profit,temp_profit_sum,timestamp,trade_id,trade_number,trades,volume
61,NaN,28195.30,1.679620e+12,NaN,NaN,2023-03-24 00:00:00,NaN,28374.30,NaN,0,...,81696702.94330860,1361.90716000,38542337.80235720,NaN,NaN,1679616000000,NaN,NaN,55358.0,2887.21044000
62,NaN,28253.00,1.679623e+12,NaN,NaN,2023-03-24 01:00:00,NaN,28318.80,NaN,0,...,64796470.41189460,1199.22863000,33870352.85455300,NaN,NaN,1679619600000,NaN,NaN,43247.0,2294.16851000
63,NaN,28186.28,1.679627e+12,NaN,NaN,2023-03-24 02:00:00,NaN,28309.99,NaN,0,...,58366428.14922550,889.81155000,25135233.48266430,NaN,NaN,1679623200000,NaN,NaN,37281.0,2066.26067000
64,NaN,28219.98,1.679630e+12,NaN,NaN,2023-03-24 03:00:00,NaN,28237.10,NaN,0,...,44359314.47674330,800.65146000,22581411.60296640,NaN,NaN,1679626800000,NaN,NaN,30487.0,1572.69903000
65,NaN,28130.41,1.679634e+12,NaN,NaN,2023-03-24 04:00:00,NaN,28254.48,NaN,0,...,56646713.28924120,943.61911000,26602825.20375980,NaN,NaN,1679630400000,NaN,NaN,36868.0,2009.41868000


In [151]:
new_cols = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'id', 'order_id', 'price', 'qty', 'commission', 'commission_asset', 'is_buyer', 'is_maker', 'datetime', 'direction', 'Wallet', 'trade_id', 'trade_number', 'temp_profit', 'profits', 'profits_ratio','temp_profit_sum']
df_sorted = df_sorted.reindex(columns=new_cols)


In [152]:
df_sorted = df_sorted.fillna(0)

df_sorted.temp_profit_sum.nunique()

16

In [153]:
duplicates = df_sorted.duplicated()
print("Nombre de lignes en double:", duplicates.sum())

Nombre de lignes en double: 0


In [154]:
# Remplacer les 0 dans la colonne 'close' par la moyenne de la ligne d'avant et d'après
df_sorted['close'].replace(0, np.nan, inplace=True)
df_sorted['close'].fillna(method='ffill', inplace=True)
df_sorted['close'].fillna(method='bfill', inplace=True)
df_sorted['close'].interpolate(method='linear', inplace=True)


### crea du statu holder pour comparer & variable a modifier pour adapter en fonction de l'historique


In [155]:
qtybase=0.0342
fiatbase=893
crypto='BTC'
fiat="USDT"

df_sorted['Holdervalue']= qtybase*df_sorted['close']

In [156]:
# création de la colonne currency

for index, row in df_sorted.iterrows():
    # Recherche du trade_id correspondant au timestamp de la ligne
    trades = trades_df[(trades_df['timestamp_start'] <= row['timestamp']) & (trades_df['timestamp_stop'] >= row['timestamp'])]
    # Si un trade est trouvé, la currency est BTC, sinon c'est USDT par défaut
    if len(trades) > 0:
        df_sorted.at[index, 'currency'] = crypto
    else:
        df_sorted.at[index, 'currency'] = fiat


In [157]:
# initialisation de la colonne qty_usdt à la somme initial d'investissement
df_sorted['qty_usdt']= fiatbase

# boucle sur chaque ligne de df_sorted
for index, row in df_sorted.iterrows():
    # récupération du timestamp de la ligne actuelle
    timestamp = row['timestamp']
    
    # boucle sur chaque ligne de df_info pour vérifier si le timestamp est compris dans une plage de temps de trade
    for info_index, info_row in df_info.iterrows():
        if info_row['timestamp_stop'] <= timestamp :
            # le timestamp est compris dans une plage de temps de trade, on calcule la quantité de BTC correspondante
            token_qty = info_row['usdt_qty']
         
            
            # on met à jour la colonne qty_btc de la ligne actuelle de df_sorted
            df_sorted.at[index, 'qty_usdt'] = token_qty
            
            # on sort de la boucle sur df_info pour ne pas continuer à vérifier les autres plages de temps
            break


In [158]:
# initialisation de la colonne qty_btc à 0
df_sorted['qty_btc'] = 0

# boucle sur chaque ligne de df_sorted
for index, row in df_sorted.iterrows():
    # récupération du timestamp de la ligne actuelle
    timestamp = row['timestamp']
    
    # boucle sur chaque ligne de df_info pour vérifier si le timestamp est compris dans une plage de temps de trade
    for info_index, info_row in df_info.iterrows():
        if info_row['timestamp_start'] <= timestamp <= info_row['timestamp_stop']:
            # le timestamp est compris dans une plage de temps de trade, on calcule la quantité de BTC correspondante
            token_qty = info_row['token_qty']
         
            
            # on met à jour la colonne qty_btc de la ligne actuelle de df_sorted
            df_sorted.at[index, 'qty_btc'] = token_qty
            
            # on sort de la boucle sur df_info pour ne pas continuer à vérifier les autres plages de temps
            break


In [159]:
#création de la colonne wallet_value
df_sorted['wallet_value']=0
for index,row in df_sorted.iterrows():
    if row ['currency']==fiat:
        value=row['qty_usdt']
        df_sorted.at[index,'wallet_value']=value
    if row ['currency']==crypto:
        value2=row['qty_btc']*row['close']
        df_sorted.at[index,'wallet_value']=value2

In [160]:


# initialisation de la colonne qty_btc à 0
df_sorted['drawdown']=0

# boucle sur chaque ligne de df_sorted
for index, row in df_sorted.iterrows():
    # récupération du timestamp de la ligne actuelle
    timestamp = row['timestamp']
    value= row['wallet_value']
    
    # boucle sur chaque ligne de df_info pour vérifier si le timestamp est compris dans une plage de temps de trade
    for info_index, info_row in df_info.iterrows():
        if info_row['timestamp_start'] <= timestamp <= info_row['timestamp_stop']:
            # le timestamp est compris dans une plage de temps de trade, on calcule la quantité de BTC correspondante
            init_valu = info_row['Wallet']
            
            
            # on met à jour la colonne qty_btc de la ligne actuelle de df_sorted
            df_sorted.at[index, 'drawdown'] = round(((value-init_valu)/init_valu) *100,2)
            
            # on sort de la boucle sur df_info pour ne pas continuer à vérifier les autres plages de temps
            break

In [161]:
df_sorted.head(50)

,timestamp,open,high,low,close,volume,id,order_id,price,qty,...,temp_profit,profits,profits_ratio,temp_profit_sum,Holdervalue,currency,qty_usdt,qty_btc,wallet_value,drawdown
61,1679616000000,28295.42,28374.30,28182.02,28195.30,2887.21044000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,964.279260,USDT,893.0,0.00000,893.000000,0.00
62,1679619600000,28195.29,28318.80,28137.93,28253.00,2294.16851000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,966.252600,USDT,893.0,0.00000,893.000000,0.00
63,1679623200000,28253.00,28309.99,28134.86,28186.28,2066.26067000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,963.970776,USDT,893.0,0.00000,893.000000,0.00
64,1679626800000,28186.28,28237.10,28158.40,28219.98,1572.69903000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,965.123316,USDT,893.0,0.00000,893.000000,0.00
65,1679630400000,28219.98,28254.48,28102.15,28130.41,2009.41868000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,962.060022,USDT,893.0,0.00000,893.000000,0.00
66,1679634000000,28130.41,28300.22,28130.40,28284.45,2593.46960000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,967.328190,USDT,893.0,0.00000,893.000000,0.00
67,1679637600000,28284.45,28335.32,28234.00,28262.83,1783.26087000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,966.588786,USDT,893.0,0.00000,893.000000,0.00
68,1679641200000,28262.84,28320.00,28162.43,28274.59,2678.97653000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,966.990978,USDT,893.0,0.00000,893.000000,0.00
69,1679644800000,28274.58,28295.72,27934.48,28034.37,5156.44522000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,958.775454,USDT,893.0,0.00000,893.000000,0.00
70,1679648400000,28034.38,28190.35,28000.00,28041.11,4506.28931000,0.000000e+00,0.000000e+00,0.00,0.00000,...,0.000000,0.000000,0.000000,0.000000,959.005962,USDT,893.0,0.00000,893.000000,0.00


### petit analyse en code pure

In [162]:
premiere_valeur = df.loc[0, 'Wallet']

# Comptage du nombre de trades positifs
nb_trades_positifs = (df['profits'] > 0).sum()
nb_trades_negatifs= (df['profits']< 0).sum()

commission_total=df['commission'].sum()
profit_total = df["profits"].sum()
profit_net=profit_total-commission_total
print('Mise de départ',premiere_valeur,'$')
print('Total des profit avant commission',profit_total,'$')
print('Total des commission',commission_total,'$')
print('Profit net après commission',profit_net,"$")

print("Il y a", nb_trades_positifs, "trades positifs dans cette historique.")
print("Il y a", nb_trades_negatifs, "trades negatifs dans cette historique.")
# Calcul de la moyenne des ratios positifs
mean_positive_ratio = df.loc[df['profits_ratio'] > 0, 'profits_ratio'].mean()

# Calcul de la moyenne des ratios négatifs
mean_negative_ratio = df.loc[df['profits_ratio'] < 0, 'profits_ratio'].mean()

print('Moyenne des trades positifs:', mean_positive_ratio,'%')
print('Moyenne des trades négatifs:', mean_negative_ratio,'%')
# Calculer la date et l'heure du premier et du dernier trade pour chaque trade_id
first_trade = df.groupby('trade_id')['timestamp'].min()
last_trade = df.groupby('trade_id')['timestamp'].max()

# Calculer la durée de chaque trade
trade_durations = last_trade - first_trade

# Calculer la durée moyenne d'un trade
mean_trade_duration = trade_durations.mean()

print("La durée moyenne d'un trade est de :", mean_trade_duration)
# Croissance
# Calculer la différence en jours entre la date actuelle et la date du premier timestamp du dataset
diff_days = (datetime.datetime.now() - pd.to_datetime(df['timestamp'], unit='ms').min()).days


croissance= profit_net/premiere_valeur*100
print('Nous avons une croissance de :',croissance,"% en seulement:",diff_days,"jours")

# Trouver le pire et le meilleur ratio de profit
worst_ratio = df['profits_ratio'].min()
best_ratio = df['profits_ratio'].max()

print("Le plus mauvais trade est :", worst_ratio,"%")
print("Le meilleur trade est :", best_ratio,"%")


# Trouver le pire drawdown
worst_drawdown = df_sorted['drawdown'].min()

print("Le pire drawdown est  :", worst_drawdown,"%")



Mise de départ 893.7354321999999 $
Total des profit avant commission 45.84011519999979 $
Total des commission 13.70401242 $
Profit net après commission 32.13610277999979 $
Il y a 6 trades positifs dans cette historique.
Il y a 9 trades negatifs dans cette historique.
Moyenne des trades positifs: 1.8077032041583536 %
Moyenne des trades négatifs: -0.6194004599274732 %
La durée moyenne d'un trade est de : 80787846.46666667
Nous avons une croissance de : 3.595706472204448 % en seulement: 28 jours
Le plus mauvais trade est : -1.690026640525967 %
Le meilleur trade est : 5.190094922808464 %
Le pire drawdown est  : -2.41 %


### Sauvegarde des fichier pour utilisation dans PowerBI

In [120]:
df_h.to_csv("Hist_btc.csv")
df.to_csv("Hist_trade_btc.csv")
df_sorted.to_csv("Hist_merge_btc.csv")